<a href="https://colab.research.google.com/github/yulita231/customer-churn-analysis/blob/main/eta_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Semua Library yang di perlukan.**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

## **Import Data dan Lihat Informasi Data**

In [ ]:
try:
    df = pd.read_csv('DATA.csv')
except FileNotFoundError:
    print("File 'DATA.csv' tidak ditemukan.")
    exit()

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Patient_ID                      25000 non-null  object
 1   Systemic Illness                18784 non-null  object
 2   Rectal Pain                     25000 non-null  bool  
 3   Sore Throat                     25000 non-null  bool  
 4   Penile Oedema                   25000 non-null  bool  
 5   Oral Lesions                    25000 non-null  bool  
 6   Solitary Lesion                 25000 non-null  bool  
 7   Swollen Tonsils                 25000 non-null  bool  
 8   HIV Infection                   25000 non-null  bool  
 9   Sexually Transmitted Infection  25000 non-null  bool  
 10  MonkeyPox                       25000 non-null  object
dtypes: bool(8), object(3)
memory usage: 781.4+ KB


,Patient_ID,Systemic Illness,Rectal Pain,Sore Throat,Penile Oedema,Oral Lesions,Solitary Lesion,Swollen Tonsils,HIV Infection,Sexually Transmitted Infection,MonkeyPox
0,P0,NaN,False,True,True,True,False,True,False,False,Negative
1,P1,Fever,True,False,True,True,False,False,True,False,Positive
2,P2,Fever,False,True,True,False,False,False,True,False,Positive
3,P3,NaN,True,False,False,False,True,True,True,False,Positive
4,P4,Swollen Lymph Nodes,True,True,True,False,False,True,True,False,Positive


## **Cek Missing Value**
Pada fitur ```Systemic Iilness``` terdapat **Missing Value** sebesar ```24%```

In [ ]:
df.isna().sum() / len(df) * 100

,0
Patient_ID,0.000
Systemic Illness,24.864
Rectal Pain,0.000
Sore Throat,0.000
Penile Oedema,0.000
Oral Lesions,0.000
Solitary Lesion,0.000
Swollen Tonsils,0.000
HIV Infection,0.000
Sexually Transmitted Infection,0.000


## **Mengisi ```Missing Value``` pada feature ```Systemic lllness``` dengan ```None``` Karena disini konteks dari feature tersebut yakni ```Menunjukkan apakah pasien memiliki penyakit sistemik lain.```**

In [ ]:
df['Systemic Illness'] = df['Systemic Illness'].fillna('None')
df['Systemic Illness'].unique()

array(['None', 'Fever', 'Swollen Lymph Nodes', 'Muscle Aches and Pain'],
      dtype=object)

## Melakukan Pre-Processing pada feature bertype data ```object```

In [ ]:
le = LabelEncoder()
df['MonkeyPox'] = le.fit_transform(df['MonkeyPox'])
for col in df.select_dtypes(include='bool').columns:
    df[col] = df[col].astype(int)
df = pd.get_dummies(df, columns=['Systemic Illness'], drop_first=True, dtype=int)

## Melakukan Feature Engineering sebelum Split Data
Fitur rekayasa dilakukan untuk membuat fitur baru dari fitur yang sudah ada. Dalam kasus ini, kita membuat fitur `Total_Symptoms` yang merupakan jumlah gejala yang dialami pasien, dan fitur interaksi `Symptoms_x_HIV` serta `Symptoms_x_STI` untuk menangkap efek gabungan dari jumlah gejala dengan status HIV dan STI.

In [ ]:
symptom_columns = [
    'Rectal Pain', 'Sore Throat', 'Penile Oedema', 'Oral Lesions',
    'Solitary Lesion', 'Swollen Tonsils', 'HIV Infection',
    'Sexually Transmitted Infection'
]
df['Total_Symptoms'] = df[symptom_columns].sum(axis=1)
print("Fitur baru 'Total_Symptoms' berhasil ditambahkan ke dataset.")

# Add interaction terms
df['Symptoms_x_HIV'] = df['Total_Symptoms'] * df['HIV Infection']
df['Symptoms_x_STI'] = df['Total_Symptoms'] * df['Sexually Transmitted Infection']
print("Interaction terms 'Symptoms_x_HIV' and 'Symptoms_x_STI' added.")

Fitur baru 'Total_Symptoms' berhasil ditambahkan ke dataset.
Interaction terms 'Symptoms_x_HIV' and 'Symptoms_x_STI' added.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 16 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Patient_ID                              25000 non-null  object
 1   Rectal Pain                             25000 non-null  int64 
 2   Sore Throat                             25000 non-null  int64 
 3   Penile Oedema                           25000 non-null  int64 
 4   Oral Lesions                            25000 non-null  int64 
 5   Solitary Lesion                         25000 non-null  int64 
 6   Swollen Tonsils                         25000 non-null  int64 
 7   HIV Infection                           25000 non-null  int64 
 8   Sexually Transmitted Infection          25000 non-null  int64 
 9   MonkeyPox                               25000 non-null  int64 
 10  Systemic Illness_Muscle Aches and Pain  25000 non-null  int64 
 11  Sy

## Melakukan Split data dan UnderSampling

In [ ]:
X = df.drop(['Patient_ID', 'MonkeyPox'], axis=1)
y = df['MonkeyPox']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Bentuk X_train sebelum resampling: {X_train.shape}")
print(f"Distribusi y_train sebelum resampling:\n{y_train.value_counts()}")

rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
print(f"\nBentuk X_train setelah resampling: {X_train_resampled.shape}")
print(f"Distribusi y_train setelah resampling:\n{y_train_resampled.value_counts()}")


Bentuk X_train sebelum resampling: (20000, 14)
Distribusi y_train sebelum resampling:
MonkeyPox
1    12727
0     7273
Name: count, dtype: int64

Bentuk X_train setelah resampling: (14546, 14)
Distribusi y_train setelah resampling:
MonkeyPox
0    7273
1    7273
Name: count, dtype: int64


## **Model Training dan Evaluasi Beberapa Algoritma**
Melatih dan mengevaluasi beberapa model klasifikasi untuk membandingkan kinerjanya.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Definisikan model-model yang akan dicoba
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Support Vector Machine": SVC(random_state=42)
}

# Loop untuk melatih dan mengevaluasi setiap model
for name, model in models.items():
    print(f"Melatih Model: {name}...")
    # Melatih model menggunakan data latih yang sudah seimbang
    model.fit(X_train_resampled, y_train_resampled)
    print(f"Pelatihan {name} selesai.")

    # Melakukan prediksi pada data uji (yang tidak diubah)
    y_pred = model.predict(X_test)

    # Menampilkan laporan evaluasi yang lengkap
    print(f"\n--- Laporan Evaluasi untuk Model: {name} ---")
    print(f"Akurasi: {accuracy_score(y_test, y_pred):.4f}")
    # classification_report memberikan rincian presisi, recall, dan f1-score
    print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))
    print("=" * 50 + "\n")

Melatih Model: Logistic Regression...
Pelatihan Logistic Regression selesai.

--- Laporan Evaluasi untuk Model: Logistic Regression ---
Akurasi: 0.6432
              precision    recall  f1-score   support

    Negative       0.51      0.63      0.56      1818
    Positive       0.76      0.65      0.70      3182

    accuracy                           0.64      5000
   macro avg       0.63      0.64      0.63      5000
weighted avg       0.67      0.64      0.65      5000


Melatih Model: Decision Tree...
Pelatihan Decision Tree selesai.

--- Laporan Evaluasi untuk Model: Decision Tree ---
Akurasi: 0.6148
              precision    recall  f1-score   support

    Negative       0.48      0.63      0.54      1818
    Positive       0.74      0.61      0.67      3182

    accuracy                           0.61      5000
   macro avg       0.61      0.62      0.61      5000
weighted avg       0.65      0.61      0.62      5000


Melatih Model: Random Forest...
Pelatihan Random Forest se

## **Hyperparameter Tuning untuk Gradient Boosting (RandomizedSearchCV)**
Melakukan pencarian acak (RandomizedSearchCV) untuk menemukan kombinasi hyperparameter terbaik untuk model Gradient Boosting.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, make_scorer

param_dist = {
    'n_estimators': np.arange(100, 500, 50),
    'learning_rate': np.arange(0.01, 0.2, 0.02),
    'max_depth': np.arange(2, 6),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 6),
    'subsample': np.arange(0.6, 1.0, 0.05),
    'max_features': ['sqrt', 'log2', None]
}

gb = GradientBoostingClassifier(random_state=42)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
}

random_search = RandomizedSearchCV(
    gb,
    param_distributions=param_dist,
    n_iter=50,
    scoring=scoring,
    refit='recall',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("Melakukan pencarian hyperparameter...")
random_search.fit(X_train_resampled, y_train_resampled)
print("Pencarian selesai.")

print("\nParameter Terbaik Ditemukan (berdasarkan Recall):")
print(random_search.best_params_)
print(f"\nSkor (Recall) Terbaik pada Cross-Validation: {random_search.best_score_:.4f}")

# Tampilkan skor rata-rata metrik lainnya untuk parameter terbaik
results = random_search.cv_results_
best_index = random_search.best_index_

print("\nSkor Rata-rata Metrik Lainnya untuk Parameter Terbaik (selama CV):")
for metric_name in scoring.keys():
    if metric_name != 'recall':
        mean_score_key = f'mean_test_{metric_name}'
        print(f"- {metric_name.capitalize()}: {results[mean_score_key][best_index]:.4f}")

# Ambil model terbaik (sudah otomatis di-refit berdasarkan 'recall' karena refit='recall')
best_gb_model = random_search.best_estimator_

## **Hyperparameter Tuning untuk Gradient Boosting (RandomizedSearchCV)**
Melakukan pencarian acak (`RandomizedSearchCV`) untuk menemukan kombinasi hyperparameter terbaik untuk model Gradient Boosting.

**Tujuan:**
Proses ini bertujuan untuk meningkatkan kinerja model dengan mencari nilai-nilai optimal untuk parameter internal model (seperti jumlah pohon, tingkat pembelajaran, kedalaman pohon, dll.). Parameter yang tepat dapat membantu model belajar pola dalam data dengan lebih efektif.

**Evaluasi Selama Tuning:**
Selama pencarian, `RandomizedSearchCV` menggunakan **Cross-Validation** (dalam kasus ini 5-fold CV, `cv=5`) pada data latih yang sudah di-resample. Untuk setiap kombinasi parameter yang dicoba (`n_iter=50`), model dilatih dan dievaluasi sebanyak 5 kali pada lipatan data latih yang berbeda.

**Metrik Penilaian (`scoring`):**
Pemilihan metrik penilaian (`scoring`) saat *hyperparameter tuning* sangat penting, terutama dalam kasus data yang tidak seimbang (imbalanced data) seperti dataset ini. Beberapa metrik yang masuk akal untuk dipertimbangkan adalah:

1.  **'recall'**: Mengoptimalkan recall bertujuan untuk meminimalkan *False Negatives* (kasus positif yang salah diprediksi sebagai negatif). Dalam konteks medis seperti deteksi penyakit, memiliki recall tinggi untuk kelas positif (MonkeyPox positif) seringkali sangat penting agar tidak melewatkan kasus yang sebenarnya.
2.  **'precision'**: Mengoptimalkan precision bertujuan untuk meminimalkan *False Positives* (kasus negatif yang salah diprediksi sebagai positif). Penting jika biaya atau konsekuensi dari prediksi positif yang salah itu tinggi.
3.  **'f1'**: F1-score adalah rata-rata harmonik dari precision dan recall. Metrik ini mencoba menyeimbangkan kedua aspek tersebut dan seringkali merupakan pilihan yang baik untuk data tidak seimbang ketika baik False Positives maupun False Negatives penting untuk diminimalkan.

Dalam kode ini, kita menggunakan `scoring='recall'` (**Optimalkan recall untuk kelas positif**) karena seringkali dalam deteksi penyakit, lebih baik mendapatkan False Positive daripada melewatkan kasus positif yang sebenarnya (False Negative).

`RandomizedSearchCV` akan memilih kombinasi parameter yang memberikan nilai metrik (`recall` dalam kasus ini) rata-rata tertinggi di seluruh lipatan cross-validation sebagai **`best_params_`**.

## **Visualisasi Confusion Matrix**
Memvisualisasikan hasil prediksi model terbaik dalam bentuk Confusion Matrix untuk melihat True Positives, False Positives, True Negatives, dan False Negatives.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model_to_visualize = best_gb_model
y_pred_to_visualize = model_to_visualize.predict(X_test) # Prediksi dari model tersebut pada data uji

# Hitung Confusion Matrix
cm = confusion_matrix(y_test, y_pred_to_visualize)

# Buat visualisasi heatmap dari Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Prediksi Model')
plt.ylabel('Aktual')
plt.show()

# Menampilkan nilai TP, FP, TN, FN secara terpisah untuk kejelasan
# Dalam confusion matrix:
# cm[0, 0] = True Negatives (TN) - Aktual Negative, Prediksi Negative
# cm[0, 1] = False Positives (FP) - Aktual Negative, Prediksi Positive (Tipe I error)
# cm[1, 0] = False Negatives (FN) - Aktual Positive, Prediksi Negative (Tipe II error)
# cm[1, 1] = True Positives (TP) - Aktual Positive, Prediksi Positive

tn, fp, fn, tp = cm.ravel()

print(f"\nHasil Confusion Matrix:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

## **Rekomendasi Keputusan Berdasarkan Hasil Model**

Berdasarkan hasil evaluasi model Gradient Boosting yang telah di-tuning, terutama melihat Confusion Matrix dan metrik Recall (yang dioptimalkan selama tuning), berikut adalah rekomendasi keputusan yang dapat diambil:

1.  **Prioritaskan Investigasi Lebih Lanjut pada Kasus yang Diprediksi Positif**:
    *   Model ini memiliki **True Positives (TP) sebesar 2228**. Artinya, dari 3182 kasus aktual positif (MonkeyPox positif) di data uji, model berhasil memprediksi 2228 di antaranya dengan benar. Ini menunjukkan bahwa model cukup baik dalam mengidentifikasi sebagian besar kasus positif.
    *   Model memiliki **False Positives (FP) sebesar 751**. Ini berarti ada 751 kasus aktual negatif (tidak menderita MonkeyPox) yang salah diprediksi oleh model sebagai positif.
    *   **Rekomendasi:** Karena model di-tuning untuk memaksimalkan *recall* (menangkap sebanyak mungkin kasus positif), model cenderung menghasilkan lebih banyak False Positives dibandingkan jika metrik lain yang dioptimalkan. Oleh karena itu, kasus-kasus yang diprediksi **positif oleh model** sebaiknya **diprioritaskan untuk investigasi atau pengujian lebih lanjut** oleh tenaga medis. Meskipun ada kemungkinan False Positive (pasien tidak benar-benar menderita), biaya dari melewatkan kasus positif (False Negative) dalam konteks penyakit menular seringkali lebih tinggi.

2.  **Waspadai Kasus yang Diprediksi Negatif tetapi Memiliki Gejala Awal**:
    *   Model memiliki **False Negatives (FN) sebesar 954**. Ini adalah kasus-kasus aktual positif (menderita MonkeyPox) yang salah diprediksi oleh model sebagai negatif. Ini adalah risiko utama ketika mengutamakan recall, tetapi masih ada kasus yang terlewatkan.
    *   Model memiliki **True Negatives (TN) sebesar 1067**. Ini berarti 1067 kasus aktual negatif berhasil diprediksi dengan benar sebagai negatif.
    *   **Rekomendasi:** Meskipun model memprediksi negatif, pasien yang menunjukkan gejala klinis yang kuat atau memiliki riwayat kontak erat dengan kasus terkonfirmasi tetap harus **dievaluasi secara klinis** oleh tenaga medis. Model ini dapat menjadi alat bantu *screening* awal, tetapi tidak boleh sepenuhnya menggantikan penilaian profesional, terutama untuk menghindari False Negatives yang signifikan.

3.  **Pertimbangkan Trade-off antara Recall dan Precision**:
    *   Model ini dioptimalkan untuk *recall* (mendeteksi positif sebanyak mungkin), yang menghasilkan *recall* yang cukup tinggi (2228 / (2228 + 954) ≈ 0.70), namun dengan *precision* yang sedikit lebih rendah (2228 / (2228 + 751) ≈ 0.75). *Precision* yang lebih rendah berarti ada cukup banyak prediksi positif yang sebenarnya adalah negatif (False Positive).
    *   **Rekomendasi:** Tingkat False Positives (751) perlu dipertimbangkan. Jika biaya (finansial, waktu, sumber daya) dari investigasi atau pengujian False Positive sangat tinggi, mungkin perlu dilakukan penyesuaian ambang batas klasifikasi model atau mempertimbangkan re-tuning model dengan metrik lain (seperti F1-score) yang menyeimbangkan recall dan precision. Namun, dengan konfigurasi saat ini yang mengutamakan recall, fokusnya adalah tidak melewatkan kasus positif.

**Kesimpulan:**

Model Gradient Boosting ini adalah alat bantu yang efektif untuk **skrining awal** atau **identifikasi risiko tinggi** MonkeyPox. Keputusan klinis akhir harus selalu didasarkan pada kombinasi hasil prediksi model, penilaian klinis pasien, riwayat medis, dan hasil tes laboratorium. Model ini membantu mengarahkan perhatian ke kasus-kasus yang paling mungkin memerlukan investigasi lebih lanjut.